In [1]:
from lib import models, graph, coarsening, utils
import numpy as np
import matplotlib.pyplot as plt
import scipy
import data_converter as dst
%matplotlib inline

In [2]:
curr_data_twitter = dst.get_twitter_data_set()
count_data = len(curr_data_twitter["data"])
print(count_data)
count_labels = len(curr_data_twitter["labels"])
print(count_labels)
assert count_data == count_labels
print(curr_data_twitter["label_values"])
count_value_labels = len(curr_data_twitter["label_values"])
for matr in curr_data_twitter["data"]:
    assert len(curr_data_twitter["data"][0]) == len(matr)

144033
144033
{'-1', '1'}


In [3]:
curr_graphs = curr_data_twitter["data"] 
labels = curr_data_twitter["labels"]
new_graphs = [scipy.sparse.coo_matrix(elem).astype(np.float32) for elem in curr_graphs]
print(new_graphs[0])

  (0, 1)	0.0003034013
  (2, 1)	0.00035130675
  (2, 3)	0.00016500772


In [4]:
n_train = count_data // 2
n_val = count_data // 10

curr_graphs = np.asarray(curr_graphs)
X_train = curr_graphs[:n_train]
X_val   = curr_graphs[n_train:n_train+n_val]
X_test  = curr_graphs[n_train+n_val:]

y_train = labels[:n_train]
y_val   = labels[n_train:n_train+n_val]
y_test  = labels[n_train+n_val:]


In [5]:
L = [graph.laplacian(A, normalized=True).astype(np.float32) for A in new_graphs]

In [6]:
params = dict()
params['dir_name']       = 'demo'
params['num_epochs']     = 40
params['batch_size']     = 1
params['eval_frequency'] = 200

# Building blocks.
params['filter']         = 'chebyshev5'
params['brelu']          = 'b1relu'
params['pool']           = 'apool1'

# Number of classes.
C = count_value_labels

print(len(L))
# Architecture.
params['F']              = [32, 64]  # Number of graph convolutional filters.
params['K']              = [20, 20]  # Polynomial orders.
params['p']              = [1, 2]    # Pooling sizes.
params['M']              = [512, C]  # Output dimensionality of fully connected layers.

# Optimization.
params['regularization'] = 5e-4
params['dropout']        = 1
params['learning_rate']  = 1e-3
params['decay_rate']     = 0.95
params['momentum']       = 0.9
params['decay_steps']    = n_train / params['batch_size']

144033


In [7]:
# X_train = np.asarray(X_train, dtype=np.float32)
y_train = np.asarray(y_train, dtype=np.float32)

# X_val = np.asarray(X_val, dtype=np.float32)
y_val = np.asarray(y_val, dtype=np.float32)

# X_train = X_train.reshape(len(X_train), 196)
# X_val = X_val.reshape(len(X_val), 196)
# print(X_train.shape)
# print(X_val.shape)
model = models.cgcnn(L, **params)
accuracy, loss, t_step = model.fit(X_train, y_train, X_val, y_val)

W0729 19:37:54.319611 140417064277824 deprecation_wrapper.py:119] From /home/anesterov/Documents/science/source/cnn_graph/lib/models.py:156: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0729 19:37:54.323754 140417064277824 deprecation_wrapper.py:119] From /home/anesterov/Documents/science/source/cnn_graph/lib/models.py:955: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0729 19:37:54.326551 140417064277824 deprecation_wrapper.py:119] From /home/anesterov/Documents/science/source/cnn_graph/lib/models.py:889: The name tf.sparse_reorder is deprecated. Please use tf.sparse.reorder instead.

W0729 19:37:54.330935 140417064277824 deprecation_wrapper.py:119] From /home/anesterov/Documents/science/source/cnn_graph/lib/models.py:898: The name tf.sparse_tensor_dense_matmul is deprecated. Please use tf.sparse.sparse_dense_matmul instead.

W0729 19:37:54.411396 140417064277824 deprecation_wrapper.py:119] From /ho

NN architecture
  input: M_0 = 14
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 14 * 32 / 1 = 448
    weights: F_0 * F_1 * K_1 = 1 * 32 * 20 = 640
    biases: F_1 = 32
  layer 2: cgconv2
    representation: M_1 * F_2 / p_2 = 14 * 64 / 2 = 448
    weights: F_1 * F_2 * K_2 = 32 * 64 * 20 = 40960
    biases: F_2 = 64
  layer 3: fc1
    representation: M_3 = 512
    weights: M_2 * M_3 = 448 * 512 = 229376
    biases: M_3 = 512
  layer 4: logits (softmax)
    representation: M_4 = 2
    weights: M_3 * M_4 = 512 * 2 = 1024
    biases: M_4 = 2


W0729 19:37:54.535473 140417064277824 deprecation_wrapper.py:119] From /home/anesterov/Documents/science/source/cnn_graph/lib/models.py:938: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0729 19:37:54.553790 140417064277824 deprecation.py:506] From /home/anesterov/Documents/science/source/cnn_graph/lib/models.py:969: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0729 19:37:54.583993 140417064277824 deprecation.py:323] From /home/anesterov/Documents/science/source/cnn_graph/lib/models.py:209: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0729 19:37:54.590046 140417064277824 deprecation_wrapper.py:119] From /home/anesterov/Documents/science/source/cnn_graph/lib/m

ValueError: Cannot feed value of shape (1, 14, 14) for Tensor 'inputs/data:0', which has shape '(1, 14)'

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 5))
ax1.plot(accuracy, 'b.-')
ax1.set_ylabel('validation accuracy', color='b')
ax2 = ax1.twinx()
ax2.plot(loss, 'g.-')
ax2.set_ylabel('training loss', color='g')
plt.show()

In [ ]:
print('Time per step: {:.2f} ms'.format(t_step*1000))

In [ ]:
res = model.evaluate(X_test, y_test)
print(res[0])